# Lab 5: Metropolis Hastings Sampling
Welcome to the fifth DS102 lab! 

The goals of this lab is to get a better understanding of the Metropolis Hastings sampling algorithm.

The code you need to write is commented out with a message "TODO: fill in". There is additional documentation for each part as you go along.


## Course Policies

**Collaboration Policy**

Data science is a collaborative activity. While you may talk with others about the labs, we ask that you **write your solutions individually**. If you do discuss the assignments with others please **include their names** in the cell below.

**Submission**: to submit this assignment, rerun the notebook from scratch (by selecting Kernel > Restart & Run all), and then print as a pdf (File > download as > pdf) and submit it to Gradescope.


**This assignment should be completed and submitted before Wednesday October 9, 2019 at 11:59 PM.** This is intentionally one day later than usual since the homework is due on Tuesday.

Write collaborator names here.

In [ ]:
%pylab inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats
import seaborn as sns

## The Function to Approximate
In this lab we will be interested in approximating the simple function $f : \mathbb{R} \rightarrow \mathbb{R}$ using Metropolis Hastings.
This is mostly for the purpose of visualization, in real life we are usually interested in functions $f : \mathbb{R^d} \rightarrow \mathbb{R}$ where $d$ is very large since this is where these MCMC type algorithms shine.

In [ ]:
def objective_f(x):
    """A simple function from R to R we wish to approximate."""
    if x < 0 or x >= 4 * np.pi:
        return 0

    return 2 + np.sin(x)

## Utility Functions for Metropolis Hastings
Now let's define some utility functions our Metropolis Hastings algorithm will use.

Recall that Metropolis Hastings views the sampling process as sequential, that is our next sample is going to depend on our previous sample. This is done through a proposal distribution $v(x'|x)$, which given the previous sample $x$ defines a probability distribution over the new sample $x'$. In this case let's pick a Gaussian proposal distribution:
$$v(x'|x) = \mathcal{N}(x'; x, \sigma),$$
we see that the distribution has mean $x$ and standard deviation $\sigma$ which we will pick later.

Recall also that we don't always accept the new proposed sample, there's a chance our new sample is simply going to be set to our old sample $x' = x$. The probability $\alpha$ of us accepting the new sample is given by
$$\min{\left(1, \frac{f(x')v(x|x')}{f(x)v(x'|x)}\right)},$$
where $f$ is the function we wish to approximate.

Below we've implemented most functions but have left the function that computes $\alpha$ empty for you to complete.

In [ ]:
def pdf_proposal_distribution(x_prime, x, sigma):
    """Given the previous sample x and new sample x_prime, computes v(x_prime|x).
    
    Parameters
    ----------
    x_prime : float
        The new sample at which we are evaluating the pdf.
    x : float
        The previous sample on which we are conditioning.
    sigma : float
        The standard deviation of our proposal function.
    
    Returns
    -------
    v : float
        The value of v(x_prime|x).
    """
    return scipy.stats.norm.pdf(x_prime, loc=x, scale=sigma)

def draw_from_proposal_distribution(x, sigma):
    """Given the previous sample x, draws a new sample from v(x_prime|x).
    
    Parameters
    ----------
    x : float
        The previous sample on which we are conditioning.
    sigma : float
        The standard deviation of our proposal function.
    
    Returns
    -------
    x_prime : float
        The generated proposed next state.
    """
    return np.random.normal(x, sigma)

def accept_probability(x_prime, x, sigma):
    """Computes the probability of accepting a new proposed sample given the previous sample.
    
    Parameters
    ----------
    x_prime : float
        The new proposed sample.
    x : float
        The previous sample from which we generated the new sample.
    sigma : float
        The standard deviation of our proposal function.
        
    Returns
    -------
    alpha : float
        The probability we will accept the proposed sample.
    """
    frac = # TODO: Fill this in.
    return np.minimum(1, frac)

def true_with_probability_alpha(alpha):
    """Randomly returns True with probability alpha, otherwise returns False."""
    return np.random.binomial(1, alpha) == 1

## Implementing Metropolis Hastings
Now that we've defined all our utility functions we can actually implement Metropolis Hastings. We've implemented most of the algorithm for you. You only need to fill in the TODOs.

In [ ]:
def metropolis_hastings(num_draws, burn_in_steps, sample_frequency, sigma):
    """Approximate objective_f using metropolis hastings.
    
    Parameters
    ----------
    num_draws : int
        The total number of samples drawn from our proposal function after burn-in.
    burn_in_steps : int
        The number of initial burn-in steps before we start considering samples.
    sample_frequency : int
        How many steps we should wait before we save the current sample.
    sigma : float
        The standard deviation of our proposal function.
        
    Returns
    -------
    samples : array of floats
        All the samples we've saved throughout the algorithm.
    all_points : array of floats
        All proposed samples (including the burn-in period), even the ones we
        didn't save.
    """
    # Initialize our return arrays.
    samples = []
    all_points = []
    
    # Initialize the first sample to be 3.
    x = 3
    for i in range(num_draws + burn_in_steps):
        all_points.append(x)
        # Propose a new point.
        x_prime = # TODO: Fill this in.
        # Compute its probability of being accepted.
        alpha = accept_probability(x_prime, x, sigma)
        # Decide whether to accept the point.
        if true_with_probability_alpha(alpha):
            x = x_prime
        # Decide whether to save the point.
        if i > burn_in_steps and i % sample_frequency == 0:
            samples.append(x)

    return np.array(samples), np.array(all_points)

## Evaluating Metropolis Hastings
Now that we have our algorithm let's see how well it approximates the objective function. We will try running Metropolis Hastings with four different values of $\sigma$ to see how it affects the performance. 

In [ ]:
samples_total = []
all_points_total = []
sigmas = [0.01, 1.0, 100.0, 10000.0]
for sigma in sigmas:
    samples, all_points = metropolis_hastings(num_draws=10000,
                                              burn_in_steps=1000,
                                              sample_frequency=10,
                                              sigma=sigma)
    samples_total.append(samples)
    all_points_total.append(samples)

Now let's plot the paths we take across the samples space for the different values of sigma.

In [ ]:
fig, ax = plt.subplots(figsize=(16,8), nrows=4)

idxs = np.arange(all_points_total[0].shape[0])
for sigma, all_points, row in zip(sigmas, all_points_total, ax):
    row.plot(idxs, all_points, zorder=2)
    row.set_title("$\sigma$=" + str(sigma))
    row.set_xlabel("TODO: Fill this in.")
    row.set_ylabel("TODO: Fill this in.")

Now let's see how well we approximated the true function for each value of sigma.

In [ ]:
fig, ax = plt.subplots(figsize=(16,16), nrows=4)

for sigma, samples, row in zip(sigmas, samples_total, ax):
    # Plot a histogram of all the samples we accepted.
    row.hist(samples, bins=100, density=True)
    # Now plot a curve of the true function, normalized to be a valid pdf, as a baseline.
    idxs = np.linspace(0, 4 * np.pi, num=1000)
    norm_const= 1 / (2 * 4 * np.pi)
    row.plot(idxs, [objective_f(i) * norm_const for i in idxs])
    
    row.set_title("$\sigma=$" + str(sigma))
    row.set_ylabel("TODO: Fill this in.")
plt.xlabel("TODO: Fill this in.")

Qualitatively, which values of $\sigma$ do you think performed best at approximating the objective function?
Can you explain why we observe the four sampling paths in the first set of plots?
How do you think performance is tied to the sampling paths we plotted above?

TODO: Fill this in.